In [1]:
import numpy as np
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

filename = Path("../data/input/20241028_test")
# dest_file= Path("..\\data\\output\\20241111\\rd4ad_resnet18_512_cc_25epoch\\version_5")
# anomaly_data = pd.read_csv(dest_file / '20241111anomalymaps.csv', header=None).values
# anomaly_scores = anomaly_data.reshape(18, 1, 512, 512)

dest_file= Path("..\\data\\output\\20241118\\rd4ad_resnet18_512_cc_70epoch\\version_0")
anomaly_data = pd.read_csv(dest_file / '20241118anomalymaps.csv', header=None).values
anomaly_scores = anomaly_data.reshape(18, 1, 512, 512)
# min_value = anomaly_scores.min()
# max_value = anomaly_scores.max()
# print(f'Min value: {min_value}, Max value: {max_value}')
# anomaly_scores = (anomaly_scores - min_value) / (max_value - min_value)

In [ ]:
# output_path = dest_file / 'hist_compare'
# output_path.mkdir(parents=True, exist_ok=True)

# plt.rcParams['font.sans-serif'] = ['Yu Gothic']

# for i in range(anomaly_scores.shape[0]):
#     plt.figure(figsize=(10, 6))
    
#     data1 = anomaly_scores[i, 0, :, :].flatten()
#     counts1, bins1, patches1 = plt.hist(data1, bins=50, color='blue', alpha=0.5, label='仮アノテーション')
#     last_nonzero_index1 = np.max(np.where(counts1 > 0))
#     max_bin_center1 = (bins1[last_nonzero_index1] + bins1[last_nonzero_index1 + 1]) / 2

#     data2 = anomaly_scores1[i, 0, :, :].flatten()
#     counts2, bins2, patches2 = plt.hist(data2, bins=50, color='orange', alpha=0.5, label='本アノテーション')
#     last_nonzero_index2 = np.max(np.where(counts2 > 0))
#     max_bin_center2 = (bins2[last_nonzero_index2] + bins2[last_nonzero_index2 + 1]) / 2

#     plt.annotate(f'Data 1 Max x: {max_bin_center1:.2f}',
#                  xy=(max_bin_center1, counts1[last_nonzero_index1]),
#                  xytext=(max_bin_center1, counts1[last_nonzero_index1] * 1.5),
#                  arrowprops=dict(facecolor='blue', arrowstyle='->'),
#                  color='blue',
#                  ha='center')

#     plt.annotate(f'Data 2 Max x: {max_bin_center2:.2f}',
#                  xy=(max_bin_center2, counts2[last_nonzero_index2]),
#                  xytext=(max_bin_center2, counts2[last_nonzero_index2] * 1.5),
#                  arrowprops=dict(facecolor='orange', arrowstyle='->'),
#                  color='orange',
#                  ha='center')

#     plt.title(f'{list(filename.glob("*.tif"))[i].stem}')
#     plt.xlabel('Anomaly Score')
#     plt.xlim(0, 0.6)
#     plt.ylabel('Frequency')
#     plt.yscale('log')
#     plt.grid(axis='y', alpha=0.75)
    
#     plt.legend(loc='upper right')
    
#     plt.savefig(output_path / f'{list(filename.glob("*.tif"))[i].stem}.png')
#     plt.close()

# print("Histograms saved to 'hist_compare' folder.")

In [ ]:
# output_dir = dest_file / 'anomaly_regions_adaptive_abstract'
# os.makedirs(output_dir, exist_ok=True)

# for i in range(anomaly_scores.shape[0]):
#     anomaly_map = anomaly_scores[i, 0, :, :]

#     anomaly_map = (anomaly_map * 255).astype(np.uint8)
#     anomaly_region = cv2.adaptiveThreshold(anomaly_map, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
#                                            cv2.THRESH_BINARY, 11, 2)

#     kernel = np.ones((5, 5), np.uint8)
#     anomaly_region = cv2.morphologyEx(anomaly_region, cv2.MORPH_OPEN, kernel)
#     anomaly_region = cv2.morphologyEx(anomaly_region, cv2.MORPH_CLOSE, kernel)

#     output_path = os.path.join(output_dir, f'{list(filename.glob("*.tif"))[i].stem}.png')
#     cv2.imwrite(output_path, anomaly_region)

# print(f'Saved anomaly region for image to {output_dir}')

In [ ]:
output_path = dest_file / 'hist'
output_path.mkdir(parents=True, exist_ok=True)

for i in range(anomaly_scores.shape[0]):
    plt.figure(figsize=(10, 6))
    data = anomaly_scores[i, 0, :, :].flatten()
    counts, bins, patches = plt.hist(data, bins=50, color='blue', alpha=0.7)
    
    last_nonzero_index = np.max(np.where(counts > 0))  
    max_bin_center = (bins[last_nonzero_index] + bins[last_nonzero_index + 1]) / 2
    
    plt.title(f'{list(filename.glob("*.tif"))[i].stem}')
    plt.xlabel('Anomaly Score')
    plt.xlim(0, 1)
    plt.ylabel('Frequency')
    plt.yscale('log')
    plt.grid(axis='y', alpha=0.75)
    
    plt.annotate(f'Last x: {max_bin_center:.2f}', 
                 xy=(max_bin_center, counts[last_nonzero_index]), 
                 xytext=(max_bin_center, counts[last_nonzero_index] * 1.5),
                 arrowprops=dict(facecolor='black', arrowstyle='->'),
                 ha='center')

    plt.savefig( output_path / f'{list(filename.glob("*.tif"))[i].stem}.png')
    plt.close()

print("Histograms saved to 'hist' folder.")

In [ ]:
output_dir = dest_file /'anomaly_regions_hist_abstract'
os.makedirs(output_dir, exist_ok=True)

thresholds = [0.36, 0.16, 0.08, 0.02]

for threshold in thresholds:
    threshold_dir = os.path.join(output_dir, str(threshold))
    os.makedirs(threshold_dir, exist_ok=True)

    for i in range(anomaly_scores.shape[0]):
        anomaly_map = anomaly_scores[i, 0, :, :]

        binary_anomaly_region = (anomaly_map > threshold).astype(np.uint8) * 255

        kernel = np.ones((5, 5), np.uint8)
        binary_anomaly_region = cv2.morphologyEx(binary_anomaly_region, cv2.MORPH_OPEN, kernel)
        binary_anomaly_region = cv2.morphologyEx(binary_anomaly_region, cv2.MORPH_CLOSE, kernel)

        output_path = os.path.join(threshold_dir, f'{list(filename.glob("*.tif"))[i].stem}.png')
        cv2.imwrite(output_path, binary_anomaly_region)

    result_image = None
    for threshold in thresholds:
        threshold_image_path = os.path.join(threshold_dir, f'{list(filename.glob("*.tif"))[i].stem}.png')
        threshold_image = cv2.imread(threshold_image_path)

        if result_image is None:
            result_image = threshold_image
        else:
            result_image = np.hstack((result_image, threshold_image))  

    result_output_path = os.path.join(output_dir, f'result_{list(filename.glob("*.tif"))[i].stem}.png')
    cv2.imwrite(result_output_path, result_image)

print(f'Saved anomaly regions and results to {output_dir}')
